In [ ]:
import h5py
from os.path import join
import numpy as np

from mssa_learning.mssa import MSSA
from mssa_learning.tools.plot import *
from mssa_learning.tools.preprocessing import *
from mssa_learning.tools.postprocessing import *

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import progressbar

In [ ]:
def extract_data(p, a, r, c):
    multiple_data = []
    for i in p:
        for j in a:
            for k in r:
                for l in c:
                    key = "P" + str(i).zfill(3) + "R" + str(k).zfill(3) + "A" + str(j).zfill(3)
                    if str(l) not in h5_data[key].keys():
                        print("Camera data not included change person, action, repetition or camera")
                    else:
                        multiple_data.append(h5_data[key][str(l)])
    return multiple_data

In [ ]:
def extract_kalman_data(p, a, r):
    multiple_data = []
    for i in p:
        for j in a:
            for k in r:
                key = "P" + str(i).zfill(3) + "R" + str(k).zfill(3) + "A" + str(j).zfill(3)
                multiple_data.append(h5_kalman[key]["thetas"])
    return multiple_data

In [ ]:
init_notebook_mode(connected=True)

In [ ]:
h5_data = h5py.File(join("..", "data", "dataset_converted" , "h5", "poses.h5"))

In [ ]:
mssa = MSSA()

### MSSA on single time serie

In [ ]:
p = [1]  # person
a = [1]  # action
r = [1]  # repetition
c = [1]  # camera

In [ ]:
multiple_data = extract_data(p, a, r, c)

In [ ]:
multiple_data = rebase(multiple_data)

In [ ]:
data = transpose(multiple_data)[0]

#### Compute principal components

In [ ]:
pc, eig_vec, eig_val = mssa.compute_principal_components(data)

In [ ]:
fig = plot_principal_components([pc])
iplot(fig)

In [ ]:
fig = plot_explained_variance(eig_val, 10)
iplot(fig)

#### Reconstruct signal

In [ ]:
rs_data = mssa.reconstruct_signal(pc, eig_vec, 10)

In [ ]:
fig = plot_timeseries(data, rs_data)
iplot(fig)

#### Calculate distance

In [ ]:
dist = []
lengths = []
for n in range(1, 25):
    rs_data = mssa.reconstruct_signal(pc, eig_vec, n)
    dist.append(mssa.signal_distance(data, np.array(rs_data)))
    rs_data = transpose([rs_data])[0]
    lengths.append(calculate_lengths(rs_data))

In [ ]:
trace = go.Scatter(
    x = np.arange(len(dist)) + 1,
    y = dist
)
iplot([trace])

In [ ]:
joint_name = "rmain"

length_data = []
for i in range(len(lengths)):
    trace = go.Scatter(
        x = np.arange(len(lengths[i][joint_name])),
        y = lengths[i][joint_name]
    )
    length_data.append(trace)
    
length = calculate_lengths(multiple_data[0])
trace = go.Scatter(
            x = np.arange(len(length[joint_name])),
            y = length[joint_name]
        )
length_data.append(trace)
iplot(length_data)

### MSSA on several repetition

In [ ]:
p = [1]  # people
a = [1]  # action
r = [1, 2]  # repetition
c = [1]  # camera

In [ ]:
multiple_data = extract_data(p, a, r, c)

In [ ]:
multiple_data = rebase(multiple_data)

In [ ]:
multiple_data = transpose(multiple_data)

In [ ]:
pcs = []
for data in multiple_data:
    pc, eig_vec,_ = mssa.compute_principal_components(data)
    pcs.append(pc[:,:10])

In [ ]:
fig = plot_principal_components(pcs)
iplot(fig)

In [ ]:
warped_pcs = dynamic_time_warping(pcs, principal_components_distance)

In [ ]:
fig = plot_principal_components(warped_pcs)
iplot(fig)

### MSSA on several people

In [ ]:
p = [1, 9, 4, 15]  # people
a = [1]  # action
r = [1]  # repetition
c = [1]  # camera

In [ ]:
multiple_data = extract_data(p, a, r, c)

In [ ]:
multiple_data = rebase(multiple_data)

In [ ]:
multiple_data = transpose(multiple_data)

In [ ]:
pcs = []
for data in multiple_data:
    pc, eig_vec,_ = mssa.compute_principal_components(data)
    pcs.append(pc[:,:10])

In [ ]:
pcs = dynamic_time_warping(pcs, principal_components_distance)

In [ ]:
fig = plot_principal_components(pcs)
iplot(fig)

### MSSA on several actions

In [ ]:
p = [1]  # people
a = [1, 5, 9, 3]  # action
r = [1]  # repetition
c = [1]  # camera

In [ ]:
multiple_data = extract_data(p, a, r, c)

In [ ]:
multiple_data = rebase(multiple_data)

In [ ]:
multiple_data = transpose(multiple_data)

In [ ]:
pcs = []
for data in multiple_data:
    pc, eig_vec,_ = mssa.compute_principal_components(data)
    pcs.append(pc[:, :10])

In [ ]:
pcs = dynamic_time_warping(pcs, principal_components_distance)

In [ ]:
fig = plot_principal_components(pcs)
iplot(fig)

### MSSA on Kalman theta

In [ ]:
h5_kalman = h5py.File(join("..", "data", "dataset_converted" , "h5", "kalman.h5"))

In [ ]:
p = [1]  # people
a = [1]  # action
r = [1]  # repetition
c = [1]  # camera

In [ ]:
multiple_data = extract_kalman_data(p, a, r)

In [ ]:
data = transpose(multiple_data)[0]

In [ ]:
pc, eig_vec, eig_val = mssa.compute_principal_components(data)

In [ ]:
fig = plot_principal_components([pc])
iplot(fig)

In [ ]:
fig = plot_explained_variance(eig_val, 10)
iplot(fig)

In [ ]:
rs_data = mssa.reconstruct_signal(pc, eig_vec, 10)

In [ ]:
fig = plot_timeseries(data, rs_data)
iplot(fig)

In [ ]:
len(rs_data)